In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from data_processing.subject_selectors import *
from data_processing.window_splitters import *
from data_processing.train_test_spliters import *
import pathlib
from os.path import join, isfile
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from models.imputer import WindowedImputer

In [ ]:



subjectSelector: SubjectSelector = SubjectSelectorList(join(pathlib.Path().resolve(), "data\\me_time_fused\\subjects"), ['1', '2'], True, True)
subjects = subjectSelector.get_subjects()
subjects

In [ ]:
windowSplitter: WindowSplitter = WindowSplitterNoOverlap(720, [np.nanmean, np.nanstd, np.nanmedian, np.nanvar, np.nanmin, np.nanmax], [np.nanmean, np.nanstd, np.nanmedian, np.nanvar, np.nanmin, np.nanmax])
windowedData: dict[str, DataFrame] = windowSplitter.split(subjects)
windowedData

In [ ]:
train, test = TrainTestSplitterOutlierPersons(1, 0.8).split(windowedData)
print(train)
print(test)

# print(len(windowedData['121']["steps_nanmin"] == 0))

In [ ]:
train_matrix = train.loc[:, test.columns != "label"].to_numpy()
test_matrix = test.loc[:, test.columns != "label"].to_numpy()

print(train_matrix)
print(test_matrix)


In [ ]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components =4).fit(train_matrix)

In [ ]:
from sklearn.metrics import roc_auc_score

score = gmm.score_samples(test_matrix)
print(score)

auc = roc_auc_score(test["label"], score)
print(f"area under the curve: {auc}")

In [ ]:
pct_threshold = np.percentile(score, 50)
print(pct_threshold)

In [ ]:
threshold = pct_threshold
labels = [1 if x > threshold else 0 for x in score]
correct = 0

for i, x in test.iterrows():
    # print(x)
    try:
        if x["label"] == labels[i]:
            correct += 1
    except:
        print(i)
        print(x)

print(f"accuracy with threshold {threshold}: {correct / len(test)}")

In [ ]:
np.nanstd([0, 0, 0, 1])

In [ ]:
import itertools
# import random
import numpy as np
l = ["aa", "b", "c"]

a = np.array(list(itertools.combinations(l, 2)))
np.random.shuffle(a)
# a = [x for x in a]
print(a)

In [ ]:
table = "| Authorized   |      Unauthorized      |  AUC |\n"
table += "| ------------- |:-------------:| -----:|\n"
table

In [ ]:
asdf = DataFrame({"a": [1, 2, 3, 4, 5], "b": [4, 5, 6, 7, 8]})

print(len(asdf))

In [ ]:
subjects: dict[str, DataFrame] = SubjectSelectorList(join(pathlib.Path().resolve(), "data\\me_time_fused\\subjects"), ['1', '2'], True, True).get_subjects()
subjects

In [ ]:
subjects: dict[str, DataFrame] = SubjectSelectorList(join(pathlib.Path().resolve(), "data\\me_time_fused\\subjects"),['1', '2'], True, True).get_subjects()
print(subjects)

In [ ]:
trainTestSplitter = TrainTestSplitterRawData(1, 0.8)
train, test = trainTestSplitter.splitBeforeWindowing(subjects)
auth, unauth = trainTestSplitter.getAuthAndUnauthIds()

print(auth)
print(unauth)

In [ ]:
# train = \
#     WindowSplitterNoOverlap(windowSize = 720, \
#                             hrTransformations = [np.nanmean, np.nanstd, np.nanmedian, np.nanvar, np.nanmin, np.nanmax], \
#                             stepTransformations = [np.nanmean, np.nanstd, np.nanmedian, np.nanvar, np.nanmin, np.nanmax]).split(train)
# test = \
#     WindowSplitterNoOverlap(windowSize = 720, \
#                             hrTransformations = [np.nanmean, np.nanstd, np.nanmedian, np.nanvar, np.nanmin, np.nanmax], \
#                             stepTransformations = [np.nanmean, np.nanstd, np.nanmedian, np.nanvar, np.nanmin, np.nanmax]).split(test)

train = \
    WindowSplitterNoOverlap(windowSize = 720, \
                            hrTransformations = [np.nanmean, np.nanmedian, np.nanvar, lambda x: np.nanmax(x) - np.nanmin(x)], \
                            stepTransformations = [np.nanmean, np.nanmedian, np.nanvar, lambda x: np.nanmax(x) - np.nanmin(x)]).split(train)
test = \
    WindowSplitterNoOverlap(windowSize = 720, \
                            hrTransformations = [np.nanmean, np.nanmedian, np.nanvar, lambda x: np.nanmax(x) - np.nanmin(x)], \
                            stepTransformations = [np.nanmean, np.nanmedian, np.nanvar, lambda x: np.nanmax(x) - np.nanmin(x)]).split(test)

train = pd.concat(train.values())
test = pd.concat(test.values())
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
print(train)
print(test)

In [ ]:

train_matrix = train.loc[:, test.columns != "label"].to_numpy()
test_matrix = test.loc[:, test.columns != "label"].to_numpy()

In [ ]:
from matplotlib.patches import Ellipse
def draw_ellipse(position, covariance, ax=None, **kwargs):
    """Draw an ellipse with a given position and covariance"""
    ax = ax or plt.gca()
    
    # Convert covariance to principal axes
    if covariance.shape == (2, 2):
        U, s, Vt = np.linalg.svd(covariance)
        angle = np.degrees(np.arctan2(U[1, 0], U[0, 0]))
        width, height = 2 * np.sqrt(s)
    else:
        angle = 0
        width, height = 2 * np.sqrt(covariance)
    
    # Draw the Ellipse
    for nsig in range(1, 4):
        ax.add_patch(Ellipse(position, nsig * width, nsig * height,
                             angle, **kwargs))
        
def plot_gmm(gmm, X, label=True, ax=None):
    ax = ax or plt.gca()
    labels = gmm.fit(X).predict(X)
    if label:
        ax.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis', zorder=2)
    else:
        ax.scatter(X[:, 0], X[:, 1], s=40, zorder=2)

    ax.axis('equal')

    w_factor = 0.2 / gmm.weights_.max()
    for pos, covar, w in zip(gmm.means_, gmm.covariances_, gmm.weights_):
        draw_ellipse(pos, covar, alpha=w * w_factor)

In [ ]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components =4).fit(train_matrix)
# plot_gmm(gmm, train_matrix)
# print(gmm.means_)
# print(gmm.covariances_)
# print(gmm.weights_)

In [ ]:
from sklearn.metrics import roc_auc_score
score = gmm.score_samples(test_matrix)
        
auc = roc_auc_score(test["label"], score)
auc

In [ ]:
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                          'foo', 'a', 'a'],
                   'B' : [1, 2, 3, 4, 5, 2, 3],
                   'C' : [2.0, 5., 8., 1., 2., 2., 3.]})

df.groupby('A').filter(lambda x: len(x) == 2)


In [ ]:
a = {
    "a": 1,
    "b": 2,
    "c": 3
}

print(type(a))
print('c' in a)


In [ ]:
n_samples = 500
np.random.seed(0)
C = np.array([[0.0, -0.1], [1.7, 0.4]])
component_1 = np.dot(np.random.randn(n_samples, 2), C)  # general
component_2 = 0.7 * np.random.randn(n_samples, 2) + np.array([-4, 1])  # spherical

X = np.concatenate([component_1, component_2])

In [ ]:

plt.scatter(component_1[:, 0], component_1[:, 1], s=0.8)
plt.scatter(component_2[:, 0], component_2[:, 1], s=0.8)
plt.title("Gaussian Mixture components")
plt.axis("equal")
plt.show()

In [ ]:
def gmm_bic_score(estimator, X):
    """Callable to pass to GridSearchCV that will use the BIC score."""
    # Make it negative since GridSearchCV expects a score to maximize
    return -estimator.bic(X)


param_grid = {
    "n_components": range(1, 7),
    "covariance_type": ["spherical", "tied", "diag", "full"],
}
grid_search = GridSearchCV(
    GaussianMixture(), param_grid=param_grid, scoring=gmm_bic_score, n_jobs=-1
)
grid_search.fit(X)

In [ ]:
df = pd.DataFrame(grid_search.cv_results_)[
    ["param_n_components", "param_covariance_type", "mean_test_score"]
]
df["mean_test_score"] = -df["mean_test_score"]
df = df.rename(
    columns={
        "param_n_components": "Number of components",
        "param_covariance_type": "Type of covariance",
        "mean_test_score": "BIC score",
    }
)
df.sort_values(by="BIC score").head()

In [ ]:
subjectsPath = join(pathlib.Path().resolve(), "data\\me_time_fused\\subjects")
subjects: dict[str, DataFrame] = SubjectSelectorList(subjectsPath,['1', '2'], True, False).get_subjects()
subjects

In [ ]:
trainTestSplitter = TrainTestSplitterRawData(1, 0.8, auth = ['1'], nonAuth = ['2'])
train, test = trainTestSplitter.splitBeforeWindowing(subjects)
auth, unauth = trainTestSplitter.getAuthAndUnauthIds()
train

In [ ]:
def groupSplitterPerTimeframe(data: DataFrame) -> np.array:
    # print(data["time"])
    first = pd.to_datetime(np.array(data["time"])[0])
    return np.array(data["time"].dt.hour // 6 + (data["time"] - first).dt.days * 4)

def groupSplitterPerHour(data: DataFrame) -> np.array:
    first = pd.to_datetime(np.array(data["time"])[0])
    return np.array(data["time"].dt.hour + (data["time"] - first).dt.days * 24)

def gmm_bic_score(estimator, X):
    """Callable to pass to GridSearchCV that will use the BIC score."""
    # Make it negative since GridSearchCV expects a score to maximize
    return -estimator.bic(X)

In [ ]:
pipe = Pipeline([("imputer", WindowedImputer(hrTransformations = [np.nanmean, np.nanmedian, np.nanvar, lambda x: np.nanmax(x) - np.nanmin(x)], groupFunction=groupSplitterPerHour)), ("gmm", GaussianMixture(n_components=4))])
train2 = pipe.fit_predict(train)
train2

In [ ]:
pipe = Pipeline([("imputer", WindowedImputer()), ("gmm", GaussianMixture())])
pipe

In [ ]:
params = {
    "imputer__hrTransformations": [[np.nanmean, np.nanmedian, np.nanvar, lambda x: np.nanmax(x) - np.nanmin(x)]],
    "imputer__stepTransformations": [None],
    "imputer__groupFunction": [groupSplitterPerHour],
    "gmm__n_components": [4]
}

In [ ]:
grid_search = GridSearchCV(
    pipe, param_grid=param_grid, scoring=gmm_bic_score
)

grid_search.fit(train)